In [107]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.cluster import KMeans
from sklearn import preprocessing
import mongo_operations as m
import re
import nltk
import itertools
import pickle

In [78]:
import pandas as pd
import numpy as np

In [79]:
def tokenize(ql):
    returning = []
    yes = set()
    for contents in ql:
        lol = nltk.word_tokenize(contents)
        returning.append(lol)
    return returning

In [80]:
def write_embeddings(filename, embeddings):
    with open(filename, 'a') as f:
        for document in embeddings:
            for embedding in document:
                f.write(str(embedding) + ' ')
            f.write('\n')

In [81]:
write_embeddings('FA19QueryEmbeddings', doc_emb_list)

In [82]:
import pickle
with open('FA19QueryEmbeddings.pickle', 'wb') as handle:
    pickle.dump(doc_emb_dict, handle)

In [95]:
def get_document_embeddings(df, model, isTokenized = False):
    corpus = df['queryText'].str.lower().tolist()
    ids=df['queryNum'].tolist()
    document_embeddings = []
    doc_emb = dict()
    c=0
    for i,document in enumerate(corpus):
        word_counts = {}
#         document = document.decode('utf-8', 'ignore')
        if not isTokenized:
#             print("Since isToeknized parameter is not set, document is being tokenized by NLTK")
            document = nltk.word_tokenize(document)
        for word in document:
            if word not in word_counts:
                word_counts[word] = 0
            word_counts[word] += 1
        this_document_embeddings = np.zeros(100)
        total_words = 0 
        for word in word_counts:
#             print(word)
            try:
                this_document_embeddings += word_counts[word] * model.wv.__getitem__(word)
                total_words += word_counts[word]
#                 print("Success! ", word, " is in vocab")
            except KeyError:
                c+=1
#                 continue
                print (word, 'not in vocabulary')
    
#         print("no. of unique words", len(word_counts))
        this_document_embeddings /= total_words
        document_embeddings.append(this_document_embeddings)
        doc_emb[ids[i]]=this_document_embeddings
    print("vocab size", c)
    return document_embeddings, doc_emb

In [96]:
def createWordEmb(df, isTokenized=False):
    queries =  df['queryText'].str.lower().tolist()
    if not isTokenized:
        print("Since isToeknized parameter is not set, queries are being tokenized by NLTK")
        queries = tokenize(queries)
    print(queries[0])
    qModel = Word2Vec(queries, sg=1, size=100, window=5, workers=12, negative=0, hs=1, sample=1e-3)
    qModel.save("word2vecQueries.model")
    qModel.wv.save_word2vec_format("QueriesNon-embedded.txt", binary=False)
    print(qModel.wv.index2word[90])
    phrasal = open("QueriesNon-embedded.txt", "r")
    vocab = []
    embeddings = []
    both = []
    new = []
    for idx, word in enumerate(qModel.wv.index2word):
            sentence = ""
            vocab.append(word)
            sentence += word+" "
            embeddings.append(qModel.wv.vectors[idx])
            for vector in qModel.wv.vectors[idx]:
                sentence += str(vector) + " "
            tup = (word, qModel.wv.vectors[idx])
            both.append(tup)
            new.append(sentence)
    m = open("ParsedFA19QueryEmbeddings.txt", "w")
    m.write(str(embeddings))
    n = open("ParsedFA19QueryPhraseEmbeddings.txt", "w")
    n.write(str(both))
    n = open("ParsedFormattedFA19Queries.txt", "w")
    n.write(str(new))
    print("Vocab len:", len(vocab))
    return qModel

In [97]:
df = pd.read_csv('./data/parsed_fa19_queries.csv')
model = createWordEmb(df)
doc_emb_list, doc_emb_dict = get_document_embeddings(df, model)
with open('ParsedFA19QueryEmbeddings.pickle', 'wb') as handle:
    pickle.dump(doc_emb_dict, handle)

Since isToeknized parameter is not set, queries are being tokenized by NLTK
['select', 'a.empname', ',', 'a.annualsalary', ',', 'b.deptname', ',', 'b.deptlocation', ',', 'a.jobtitle', 'from', 'employee', 'a', 'natural', 'join', 'department', 'd', 'where', '(', 'b.deptlocation', '=', "'dubai", "'", 'or', 'b.deptname', '=', "'sales", "'", ')', 'and', '(', 'a.annualsalary', '>', '=', '7000', 'and', 'a.annualsalary', '<', '=', '100000', ')', 'and', '(', 'a.empname', 'like', "'_a", '%', "'", 'or', 'a.empname', 'like', "'_i", '%', "'", ')', 'order', 'by', 'a.empname', 'asc', ',', 'b.deptname', 'desc']
jobtitle
Vocab len: 454
100000 not in vocabulary
/12 not in vocabulary
=10000 not in vocabulary
100000 not in vocabulary
70000 not in vocabulary
substr not in vocabulary
a.deptname= not in vocabulary
b.annualsalary/12 not in vocabulary
=10000 not in vocabulary
annualsalary/12 not in vocabulary
e.annualsalary not in vocabulary
e.jobtitle not in vocabulary
b.annualsalary/12 not in vocabulary
=700

i.a not in vocabulary
but not in vocabulary
combined not in vocabulary
with not in vocabulary
b.attribute not in vocabulary
ask not in vocabulary
something not in vocabulary
you not in vocabulary
can not in vocabulary
anything not in vocabulary
clause not in vocabulary
that not in vocabulary
is not in vocabulary
use not in vocabulary
employees not in vocabulary
find not in vocabulary
this not in vocabulary
will not in vocabulary
help not in vocabulary
see not in vocabulary
per not in vocabulary
department='ece not in vocabulary
department='cs not in vocabulary
sa.netid not in vocabulary
ss not in vocabulary
cc not in vocabulary
ee.netid not in vocabulary
ee.crn not in vocabulary
cc.crn not in vocabulary
cc.department='cs not in vocabulary
/*from not in vocabulary
emp.departmentid not in vocabulary
dept.departmentid*/ not in vocabulary
enrollments.netid=students.netid not in vocabulary
'and not in vocabulary
stu not in vocabulary
s.department= not in vocabulary
h not in vocabulary
h.dep

b.comission not in vocabulary
mul not in vocabulary
e3.managerid not in vocabulary
e3 not in vocabulary
e2.salary*e2.comission not in vocabulary
salary_after_commision not in vocabulary
benchmark not in vocabulary
*comission not in vocabulary
jobtitle= not in vocabulary
jobtitle='it not in vocabulary
/* not in vocabulary
another not in vocabulary
attempt not in vocabulary
: not in vocabulary
*/ not in vocabulary
'general not in vocabulary
'operations not in vocabulary
managersalary not in vocabulary
e2.empname not in vocabulary
e2.salary* not in vocabulary
jobtitle= not in vocabulary
after not in vocabulary
commision not in vocabulary
list not in vocabulary
managers not in vocabulary
e2.salary*e2.comission not in vocabulary
jobtitle= not in vocabulary
m.salary*m.comission not in vocabulary
m.jobtitle not in vocabulary
a.salary* not in vocabulary
emp2.salary not in vocabulary
emp2 not in vocabulary
.comission not in vocabulary
emp2.jobtitle not in vocabulary
comission*salary not in voca

In [24]:
print(len(doc_emb_dict))

1418


In [110]:
print(type(doc_emb_dict[1]))

<class 'numpy.ndarray'>


In [111]:
#Writing embeddings to mongo
mongo = m.MongoDB("Queries")
for key in doc_emb_dict:
    val = doc_emb_dict[key].tolist()
    mongo.update_one_query(({'queryNum': key}, {'embedding': val}))

({'queryNum': 1}, {'embedding': [-0.10443454231211885, 0.03871216793300309, -0.01633394967581405, 0.04061261953816551, -0.0015831137014949907, 0.04691746959187945, 0.19781399982385947, -0.019764644200684595, -0.15301697890533775, 0.11338640205165157, 0.032955083018932185, 0.2724700644001609, 0.08535128648652406, 0.14450573275384845, -0.08474225367678971, -0.008088625868263304, 0.092657456937872, -0.019022772264224094, -0.022652656725439868, -0.02483887153440995, -0.09213309631240173, -0.20313318250280973, -0.020843970940494148, -0.13620327155059966, 0.07651301108483897, -0.044605314517851734, -0.06078109736401649, -0.05873978852492864, 0.008013989409950913, 0.18372817213845546, -0.0432217700223698, 0.08376205700342773, 0.005034865693143401, -0.12244406682210135, -0.29532661093551604, -0.04758218585773081, -0.01817080925875267, 0.11259202506454265, 0.15733912529149016, 0.2047254461413402, -0.1578353802982687, 0.05320155561031377, 0.006308523663243309, 0.12023628338194284, -0.22108499620

Query 5eaf1fef334f483dd6bb9e38 has been updated
({'queryNum': 6}, {'embedding': [-0.093939849977883, 0.07642394605522546, 0.0325258613205873, 0.03360385895491792, 0.07623836021297253, 0.09540785384865907, 0.24031126972001332, 0.015872362886483853, -0.2056217653820148, 0.12533389940284767, 0.0760491967201233, 0.2726068772375584, 0.05844857068015979, 0.06336133945733309, -0.04271656967556248, -0.012221016412457595, 0.05415024270231907, -0.008928372847059598, -0.0503939922612447, 0.09286225886764721, -0.10814050395901387, -0.11539634534945854, -0.07776365847541736, -0.051663188736599225, 0.13805601125439773, 0.01730997132567259, -0.07647509330238861, -0.09797678907903341, -0.011522640603093, 0.2061401246688687, -0.05385668978381616, 0.024471518827172425, 0.14915684804391977, -0.03184066365401332, -0.2672534133379276, -0.11107179241684767, -0.04556645404977294, 0.12341055268278489, 0.132449913619516, 0.17594342070560043, -0.11257789370138198, 0.06950318751713404, 0.07654466230708819, 0.119

Query 5eaf1fef334f483dd6bb9e3c has been updated
({'queryNum': 10}, {'embedding': [-0.06933542061597109, 0.04746951752536888, -0.005865153008654936, -0.03893464385295537, 0.0566192495761745, 0.058686126273737024, 0.1871214213211145, -0.01403983993761575, -0.18946129934334044, 0.12013937774767626, 0.08055624899579518, 0.27854445715671156, 0.06243985367299461, 0.10969446333987054, -0.024924652393796105, -0.0188010592611312, 0.0671726145795477, 0.028808392465302247, -0.015013919392628456, 0.06915062225472404, -0.16076969966959598, -0.2169732636591392, -0.010959069439168297, -0.09536692053913626, 0.09187743839210094, -0.028209033324869712, -0.06193537255679605, -0.061292956541500875, -0.018987403654340488, 0.19470246690811951, -0.04764631578052388, 0.0717838123829952, 0.12525647827563113, -0.038430531863679195, -0.21978916889473574, -0.019143177324961593, 0.06819383742343356, 0.11424145914280592, 0.10011516534375833, 0.13008391478363035, -0.06602139869850795, 0.1100522631278901, 0.041174557

Query 5eaf1fef334f483dd6bb9e43 has been updated
({'queryNum': 17}, {'embedding': [-0.10149627795555827, 0.07331126858127501, 0.015619719231670553, 0.038782142511935846, 0.02040982112344919, 0.08159806714816527, 0.19493224195232897, 0.00190058167120724, -0.17555211275590188, 0.12371710356974015, 0.06684588511107546, 0.2693591315976598, 0.08315735338537981, 0.1401678844298603, -0.07595579210443026, -0.022210065240153308, 0.09232239498559272, 0.003015953257228389, -0.0670366505104484, -0.010408421002935167, -0.10250400825206077, -0.17596645647074294, -0.04244656791684754, -0.10516264629691388, 0.08916676373249202, 0.0024604434711915073, -0.06121929764931062, -0.059140915615540565, 0.020145868769649303, 0.2023895783490981, -0.04916622840877002, 0.052845764284332596, 0.06281481562252862, -0.10228927041911944, -0.2820163999091495, -0.0766019714629334, -0.002998292650745222, 0.1382756173384912, 0.14962030216261293, 0.2000908602071418, -0.14011328447447863, 0.038545101500031625, 0.043716073431

Query 5eaf1fef334f483dd6bb9e47 has been updated
({'queryNum': 21}, {'embedding': [-0.0650099705551462, 0.052791345932777844, 0.006141859138834065, -0.04511836323694422, 0.05268873571386111, 0.049957737198163724, 0.1878342735420527, -0.019193068082476485, -0.19732042074460407, 0.12067419725695047, 0.09844680747081494, 0.2956649849897829, 0.07265669208598034, 0.12446557847654512, -0.030011426448693562, -0.029132665191166873, 0.06566938189468507, 0.02923332937443147, -0.0202241828986283, 0.08159836887679031, -0.14785817618770847, -0.2249436223686769, -0.012386557180434465, -0.08935314206534932, 0.11052095483795836, -0.027263682496188015, -0.07755407254645151, -0.05772079819620683, -0.03058317072432617, 0.1873745047808464, -0.05134524059670592, 0.0411987121955588, 0.13588063634032832, -0.05157902286837584, -0.20058809240059605, 0.0006705886796759121, 0.04835390001695988, 0.1273440299876805, 0.09838383199229579, 0.1243845269260607, -0.07956079398643161, 0.10671524717954212, 0.06063379491839

...

Query 5eaf2015334f483dd6bba3a3 has been updated
({'queryNum': 1393}, {'embedding': [-0.09807010181248188, -0.09255481587024406, -0.14521860232343897, 0.06142294674646109, 0.04740294168004766, -0.03666827391134575, 0.05381017760373652, -0.018900159426266328, -0.05686467273335438, 0.010338265594327822, -0.006822278839536011, 0.14193573512602597, 0.09536586066678865, 0.0022613562614424154, -0.12647226254921407, -0.07184052548836917, 0.04672035563271493, -0.04565337978419848, 0.18537600006675348, 0.10189487301977351, 0.07891363717499189, -0.28043170750606805, -0.04626779240788892, -0.07444405349087901, -0.025914113502949476, -0.23674705481607816, -0.01923757151234895, -0.1955830033057282, 0.01788541063433513, 0.2076386846601963, 0.07685657820547931, 0.31557956244796515, 0.059422792146506254, -0.011654847898171283, -0.1783995635341853, -0.266936844564043, 0.15591436965041794, -0.08091958065051585, 0.09297519821848255, 0.03311164685874246, 0.06361050839768723, 0.30680571880657226, -0.0054232

Query 5eaf2015334f483dd6bba3aa has been updated
({'queryNum': 1400}, {'embedding': [-0.09368567248047502, -0.0792970878796445, -0.11349226706833751, 0.03560459280938462, 0.042998253643788675, -0.04336269420606119, -0.012310724222549686, -0.04777136896909387, -0.07395380307678823, 0.004657232961444943, 0.02509430354392087, 0.16269655090204818, 0.10775461047887802, 0.004785143390849784, -0.12506677743254435, -0.09049329961891528, 0.04151886853354948, -0.08048709274994002, 0.1952814739887361, 0.12055692787247675, 0.09721971351515364, -0.22105986162744187, -0.03944783167982543, -0.10821097565780359, -0.004630826896539441, -0.2696624573458124, -0.03497985215788638, -0.24929308615348958, -0.04147880310537638, 0.21954960955513847, 0.0772578663158196, 0.2899077083501551, 0.03422031330841559, -0.014486591952542463, -0.16707327920529577, -0.25114439662407945, 0.09029620644394998, -0.09444277864639405, 0.06506932647553859, 0.026438628202649177, 0.07223687262308819, 0.3267158911459976, 0.058784431

Query 5eaf2015334f483dd6bba3b0 has been updated
({'queryNum': 1406}, {'embedding': [-0.0592075055132487, -0.044347974338701794, -0.13471636941124285, 0.047659153917006085, 0.02453756788080292, -0.023884212831035256, 0.07365738873236946, -0.019704907334276607, -0.06692648506058115, 0.03703615824425859, 0.012741916347295046, 0.17565850107764294, 0.055614711450678964, -0.011910664632783405, -0.058920979466555376, -0.07236236214105572, 0.018342056710805212, -0.05036657291930169, 0.24402943093861854, 0.14155567730111734, 0.04348469838233931, -0.18523244333586522, -0.06329393566453032, -0.12451636103547312, 0.0232959768202688, -0.21513136409755265, -0.03755683874312256, -0.24606180377304554, 0.016418466327844987, 0.20855150970497302, 0.06129282104250576, 0.2753180503579123, 0.0332603293354623, -0.04935343156103045, -0.21683874247329576, -0.27243686001747847, 0.07868876141895141, -0.060826645764921396, 0.1018388009397313, 0.06988246777161423, 0.03406059754862716, 0.24989780730434827, -0.01132

Query 5eaf2016334f483dd6bba3b7 has been updated
({'queryNum': 1413}, {'embedding': [-0.0683599661922816, -0.031004435523892895, -0.14110963217293224, 0.03636748417083061, -0.001850533750698422, -0.0005559408755013437, 0.016430423397457962, -0.036162727286644054, -0.021151259089961197, -0.013291826782127222, 0.04113215371740587, 0.12866051714500468, 0.07351743954826485, -0.010406222744760189, -0.09590533309418595, -0.1335213169716581, -0.022649482569911263, -0.05045922790569338, 0.22979262521998448, 0.1496878293427554, 0.07672457368762205, -0.23143925574241261, -0.04024767944109485, -0.15114854099087868, 0.019892942781249683, -0.26097893342375755, -0.028926866383037785, -0.2880906914219712, 0.042047768266815125, 0.18947051838040352, 0.046823232237136726, 0.3149044747379693, -0.0073752314289749574, -0.042360033924606716, -0.1907235158212257, -0.22964731919946094, 0.07309340679961623, -0.09480896992891123, 0.09572237052700737, 0.04786796862203063, 0.05557397035811322, 0.2700176929885691, 

Query 5eaf2016334f483dd6bba3bb has been updated
({'queryNum': 1417}, {'embedding': [-0.07878248318022973, -0.08488197700196021, -0.1306053372855122, 0.008201622215376512, 0.0599775027634727, -0.031426864147589016, 0.009009659139288438, -0.058192134406921024, -0.0651504519219334, 0.01580526732613106, 0.016636545291623554, 0.17099613978250608, 0.08953330132204133, -0.01996400670425312, -0.11967431475735597, -0.09147935159932319, 0.04597758582314929, -0.0362549685696895, 0.1964484401852698, 0.13554149166353652, 0.08329194648241675, -0.24192488022349976, -0.06905786509348734, -0.11474981615823265, 0.022049395719895493, -0.22937296277755317, -0.06719094713696756, -0.2341097293270601, -0.0444368904305471, 0.20325707231421727, 0.08607698244520941, 0.26617869563602115, 0.0548947550155021, -0.033098994036884724, -0.18794306528729363, -0.25147575784373927, 0.1153552400426486, -0.07172873580979335, 0.10877774287065542, 0.035940865331606286, 0.06108000677871845, 0.30460587546632095, 0.009516637419

In [112]:
print(mongo.read_one_query({'queryNum': just_one[0][0][0]}))

NameError: name 'just_one' is not defined

## Trying replacement of non-dictionary words - PLS ignore

In [26]:
fv = pd.read_csv('./data/fruits_and_veggies.csv')

In [27]:
fv.head()

,Name
0,Açaí
1,Ackee
2,Apple
3,Apricot
4,Avocado


In [64]:
names =  fv['Name'].str.lower().tolist()
new = names + names + names + names + names
# str1 = " "
# namestr = str1.join(names)
lol = tokenize(new)
# print("Toek:   ", lol)
model = Word2Vec(lol, sg=1, size=2, window=5, workers=12, negative=0, sample=1e-3)
print(len(names))
c=0
inDict = []
for name in names:
    try:
        model.wv.__getitem__(name)

    except KeyError:
        c+=1
        print("in dict:", name)
#         print(name, "Not in dict")
word_vectors = model.wv
print(len(word_vectors.vocab))
print(word_vectors.vocab.keys())
print(c)

248
in dict: black sapote
in dict: cactus pear
in dict: crab apple
in dict: chico fruit
in dict: goji berry
in dict: grewia asiatica
in dict: hala fruit
in dict: japanese plum
in dict: juniper berry
in dict: galia melon
in dict: miracle fruit
in dict: blood orange
in dict: purple mangosteen
in dict: salal berry
in dict: star apple
in dict: star fruit
in dict: surinam cherry
in dict: ugli fruit
in dict: white currant
in dict: white sapote
in dict: bell pepper
in dict: chile pepper
in dict: corn kernel
in dict: alfalfa sprouts
in dict: azuki beans
in dict: bean sprouts
in dict: black beans
in dict: black-eyed peas
in dict: borlotti bean
in dict: broad beans
in dict: green beans
in dict: kidney beans
in dict: lima beans 
in dict: mung beans
in dict: navy beans
in dict: pinto beans
in dict: runner beans
in dict: split peas
in dict: soy beans
in dict: brussels sprouts
in dict: savoy cabbage
in dict: red cabbage
in dict: bok choy
in dict: collard greens
in dict: mustard greens
in dict: new z